In [1]:
import pickle
import pandas as pd

post_info = pickle.load( open( "save.p", "rb" ) )

In [2]:
post_info

,display_url,caption,comments,likes,labels
0,https://instagram.fftw1-1.fna.fbcdn.net/vp/801...,Photo by Erika Larsen @erikalarsen888 | Roxane...,360,47452,"[Clothing, Dress shirt, Sleeve, Shirt, Formal ..."
1,https://instagram.fftw1-1.fna.fbcdn.net/vp/8c4...,Photo by David Guttenfelder @dguttenfelder // ...,336,85229,"[Bicycle, Pink, Bicycle wheel, Vehicle, Cyclin..."
2,https://instagram.fftw1-1.fna.fbcdn.net/vp/764...,Photo by George Steinmetz @geosteinmetz | A ca...,758,292916,"[Yellow, Sky, Sand, Dune, Landscape, Heat, Aeo..."
3,https://instagram.fftw1-1.fna.fbcdn.net/vp/527...,Photo by Maggie Steber @maggiesteber | Veteran...,254,98153,"[Design, Room, Photography, Material property,..."
4,https://instagram.fftw1-1.fna.fbcdn.net/vp/a33...,Photo by Anastasia Taylor-Lind @anastasiatl | ...,646,132572,"[Room, Photography, Sitting]"
...,...,...,...,...,...
942,https://instagram.fftw1-1.fna.fbcdn.net/vp/966...,Photo by Brian Skerry @brianskerry | A threshe...,5255,531996,"[Fish, Shark, Blue, Water, Great white shark, ..."
943,https://instagram.fftw1-1.fna.fbcdn.net/vp/24b...,Photo by Stephen Wilkes @stephenwilkes | It is...,3792,826978,"[Elephant, Elephants and Mammoths, Mammal, Ter..."
944,https://instagram.fftw1-1.fna.fbcdn.net/vp/8a1...,Photo by Michael Yamashita @yamashitaphoto | O...,2070,754997,"[Nature, Natural landscape, Reflection, Wilder..."
945,https://instagram.fftw1-1.fna.fbcdn.net/vp/96f...,"Photo by Joel Sartore @joelsartore | Hurry, be...",1292,448321,"[Sky, Blue, Tree, Nature, Night, Branch, Woody..."


In [3]:
post_info['likes_normalized'] = post_info['likes']/post_info['likes'].max()

In [4]:
post_info['comments_normalized'] = post_info['comments']/post_info['comments'].max()

In [5]:
post_info['engagement_score'] = .4*post_info['likes_normalized']+.6*post_info['comments_normalized']

In [6]:
engagement_median = post_info['engagement_score'].median()

def binary_engage(score):
    if score > engagement_median:
        return 1
    else:
        return 0

In [7]:
post_info['engagement_binary'] = post_info['engagement_score'].map(binary_engage)

In [8]:
post_info.engagement_binary.value_counts()

0    474
1    473
Name: engagement_binary, dtype: int64

In [28]:
regression = post_info[['labels', 'engagement_binary']]

In [24]:
labels = list(set(post_info['labels'].sum()))

In [37]:
for label in labels:
    regression[label] = regression['labels'].apply(lambda x: 1 if label in x else 0 )

In [38]:
regression

,labels,engagement_binary,Bank,Coast,Rhesus macaque,Classic,Cycling,Boating,Dog,Ocean,...,Bald eagle,Erg,Honeymoon,Photography,Flag,Nativity scene,Tide,Crane,Atmosphere,Mud
0,"[Clothing, Dress shirt, Sleeve, Shirt, Formal ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Bicycle, Pink, Bicycle wheel, Vehicle, Cyclin...",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[Yellow, Sky, Sand, Dune, Landscape, Heat, Aeo...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[Design, Room, Photography, Material property,...",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,"[Room, Photography, Sitting]",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,"[Fish, Shark, Blue, Water, Great white shark, ...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
943,"[Elephant, Elephants and Mammoths, Mammal, Ter...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
944,"[Nature, Natural landscape, Reflection, Wilder...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
945,"[Sky, Blue, Tree, Nature, Night, Branch, Woody...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [42]:
X = regression.drop(['labels', 'engagement_binary'], axis = 1)
y = regression[['engagement_binary']]

In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [46]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
log_reg.score(x_test, y_test)

0.7257383966244726

In [53]:
from sklearn import metrics

predictions = log_reg.predict(x_test)

metrics.confusion_matrix(y_test, predictions)

array([[93, 33],
       [32, 79]], dtype=int64)

In [55]:
predictions

array([1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0], dtype=int64)

In [67]:
confusion = pd.DataFrame(zip(y_test.engagement_binary, predictions), columns=['actual', 'predicted'])
pd.crosstab(confusion.actual, confusion.predicted)

predicted,0,1
actual,,
0,93,33
1,32,79
